# Slider Example
This example will allow the user to use sliders to interact with the ISR spectra. Currently this notebook creates two images. The first is a dual plot of the ACF and Power spectra expected for 100% O+ ionosphere and k = 18.5 with ion and electron temperature from the sliders. The second image is an image of the spectra that varies over wave number and an ion temperature derived from the sliders.

In [1]:
import numpy as np
import matplotlib as mpl
import scipy.fftpack as scfft
import scipy.constants as spconst

import matplotlib.pylab as plt
import seaborn as sns
sns.set_style("white")
sns.set_context("notebook")
#
from ISRSpectrum import Specinit
import ipywidgets 
from IPython.display import display
#%matplotlib inline
#%matplotlib notebook
%matplotlib widget
def plot1dspec(Ti,Tr):
    """Plot an IS spectra and ACF given ion and electron temperatures.
    Will plot a spectra and ACF over a sampling rate of 50 kHz 256 samples with a center frequency of 440 MHz
    
    Parameters
    ----------
    Ti : float
        Ion temperture in degrees K.
    Tr : float
        Ratio of electron temperature over ion temperature.
    """

    Te= Tr*Ti
    databloc = np.array([[1e11, Ti], [1e11,Te]])
    
    # Select Number of points and sampling frequency in the spectra
    nspec=256
    spfreq=50e3
    cfreq = 440e6
    kvec = 4*spconst.pi*cfreq/spconst.c
    ISpec_ion = Specinit(centerFrequency=cfreq, nspec=nspec, sampfreq=spfreq,dFlag=True)
    
    species=['O+','e-']
    ylim=[0,1.4]
    ylim2=[-.5,1.4]
    flims=np.array([3.03e6,3.034e6])
    
    
    fion,ionline= ISpec_ion.getspecsep(databloc,species)
    
    acf=scfft.ifft(scfft.ifftshift(ionline)).real
    tau=scfft.ifftshift(np.arange(-np.ceil((float(nspec)-1)/2),np.floor((float(nspec)-1)/2)+1))/spfreq
    
    

    if 'fig' in locals():
        fig.close()
        del fig
        del ax
        
    fig,ax = plt.subplots(1,2,sharey=False,facecolor='w')
    

    l1=ax[0].plot(fion*1e-3,ionline/ionline.max(),'-',lw=3)[0]
    sns.despine()

    ax[0].set_xlim([-15,15])
    ax[0].spines['right'].set_visible(False)

    ax[0].set_xlabel('Frequency (kHz)',fontsize=14)
    kstr = '{:0.1f}'.format(kvec)
    ax[0].set_title('Spectra K={:0.1f}'.format(kvec),fontsize=18)
    ax[0].set_ylabel('Normalized Magnitude',fontsize=14)
    
    ax[0].set_ylim(ylim)
    
    
    
    l1=ax[1].plot(tau[:64]*1e6,acf[:64]/acf[0],'-',lw=3)[0]
    sns.despine()

    ax[1].set_xlim([0,480])
    ax[1].spines['right'].set_visible(False)

    ax[1].set_xlabel('Lag in Microseconds ',fontsize=14)
    ax[1].set_title('ACF K={:0.1f}'.format(kvec),fontsize=18)
    ax[1].set_ylabel('Normalized Magnitude',fontsize=14)
    
    ax[1].set_ylim(ylim2)
    plt.tight_layout()
    plt.show()


## ACF and Spectra
This will show a plot of the spectra and ACF with sliders for the electron and ion temperatures. The frequency of the radar is 440 MHz carrier frequency (UHF).

In [2]:

slideTi = ipywidgets.FloatSlider(
    value=1000.,
    min=500.,
    max=5000.,
    step=500.,
    description='Ti in K:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    slider_color='white'
)
slideTr = ipywidgets.FloatSlider(
    value=1.,
    min=1.,
    max=8.,
    step=0.25,
    description='Tr: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    slider_color='white'
)

%matplotlib inline
w=ipywidgets.interactive(plot1dspec,Ti=slideTi,Tr=slideTr)
output = w.children[-1]
output.layout.height = '800px'
output.layout.width= '100%'
display(w)

interactive(children=(FloatSlider(value=1000.0, continuous_update=False, description='Ti in K:', max=5000.0, m…

## Spectra Image Varying by Wave Number
A plot will be created showing how the IS spectra will vary with wave number using the ion temperature from the slider.

In [3]:
slide1 = ipywidgets.FloatSlider(
    value=1000.,
    min=500.,
    max=5000.,
    step=500.,
    description='Ti in K:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    slider_color='white'
)

def plot2dspec(Ti):
    """Create a 2-D plot of spectra from wave number and frequency.
    
    Parameters
    ----------
    Ti : float
        Ion temperture in degrees K.
    """
    databloc = np.array([[1e11,Ti],[1e11,2.5e3]])
    species=['O+','e-']
    newcentfreq  = 449e6 +np.linspace(-200,550,250)*1e6
    k_all= 2*2*np.pi*newcentfreq/spconst.c
    k_lims=np.array([k_all.min(),k_all.max()])
    freq_lims=np.array([newcentfreq.min(),newcentfreq.max()])
    oution = []
    for i,if_0 in enumerate(newcentfreq):
        ISpec_ion = Specinit(centerFrequency = if_0, nspec=256, sampfreq=50e3,dFlag=False)
        fion,ionline= ISpec_ion.getspecsep(databloc,species)
        oution.append(ionline)
    oution=np.array(oution)
    
    
    F,K_b=np.meshgrid(fion,k_all)
    
    fig,ax = plt.subplots(1,1,sharey=True, figsize=(6,4),facecolor='w')
    
    l1=ax.pcolor(F*1e-3,K_b,oution/oution.max(),cmap='viridis')
    cb1 = plt.colorbar(l1, ax=ax)
    ax.set_xlim([-25,25])
    ax.set_ylim(k_lims)
    ax.set_xlabel('Frequency (kHz)',fontsize=14)
    ax.set_title(r'$\langle|n_e(\mathbf{k},\omega)|^2\rangle$',fontsize=18)
    ax.set_ylabel(r'$|\mathbf{k}|$ (rad/m)',fontsize=14)
%matplotlib inline
w = ipywidgets.interactive(plot2dspec, Ti=slide1)
output = w.children[-1]
output.layout.height = '1200px'
display(w)

interactive(children=(FloatSlider(value=1000.0, continuous_update=False, description='Ti in K:', max=5000.0, m…